In [4]:
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
from openpyxl import load_workbook

load_dotenv()

client = OpenAI() # defaults to os.environ.get('OPENAI_API_KEY')

# Load the Excel file
file_path = './file_folder/Data_Extraction_Profitability.xlsx'
df = pd.read_excel(file_path)
# df = df.head(105)

# Set up your OpenAI API key
# openai.api_key = os.environ.get('OPENAI_API_KEY')

def extract_value(note, task_description, operational_guidelines, examples):
    prompt_template= f"""You are an expert data extractor. Your task is to {task_description}

    Operational Guidelines:
    {operational_guidelines}

    Examples:
    {examples}

    Here is the note: "{note}"
    Return ONLY the resulting length-2 JSON array.
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": prompt_template}]
        #, temperature?
    )

    response_text = response.choices[0].message.content.strip()
    print("API response: ", response_text, "to note: ", note)

    # Ensure the response is a valid JSON
    if response_text.startswith("```") and response_text.endswith("```"):
        response_text = response_text[3:-3].strip()
    response_text = response_text.strip("json\n")
    print('clen',response_text)

    # Parse the JSON response
    try:
        extracted_values = json.loads(response_text)
        return extracted_values
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON response: {e}")
        return [['?','?']]*5
    
def process_notes(df, column_name, task_description, operational_guidelines, examples):
    values, confidences = [], []
    for note in df[column_name]:
        print('note: ', note)
        extracted_value, confidence = extract_value(note, task_description, operational_guidelines, examples)
        values.append(extracted_value)
        confidences.append(confidence)
    return values, confidences

def main():
    task_description = "extract the profitability value from the following note."
    operational_guidelines = """
    1. Convert the value to one of the following categories: "Profitable", "Not Profitable", "Break Even", "Unknown", or "Blank". 
    2. If the value is a positive number or percentage, it is profitable.
    3. BE stands for "Break Even". CF stands for Cash Flow; positive Cash Flow means "Profitable".
    3. If the input is an empty string or nan, return "Blank".
    4. If the value is vague, return "Unknown".
    5. Provide a confidence level in the range (0,1] for the extracted value. The confidence level should reflect how certain you are of your converted value.
    6. Return ONLY a length-2 JSON array with the extracted value and confidence level.
    """
    
    examples = """
    Note: "Burning $5mm" -> ["Not Profitable", 0.9]
    Note: " " -> ["Blank", 1.0]
    Note: "BE by Dec'20" -> ["Break Even", 0.9]
    Note: "" -> ["Blank", 1.0]
    Note: "Yes" -> ["Profitable", 1.0]
    Note: "burning" -> ["Not Profitable", 0.9]
    Note: "50% EBITDA" -> ["Profitable", 1.0]
    Note: "burning at corporate level" -> ["Not Profitable", 0.9]
    Note: "50% EBITDA" -> ["Profitable", 1.0]
    Note: "-" -> ["Unknown", 1.0]
    Note: "?5M EBITDA 2024" -> ["Profitable", 0.7]
    Note: "CF+" -> ["Profitable", 1.0]
    Note: "Self-financed" -> ["Profitable", 0.8]
    Note: "61% GM, ?6M 2021E Adj. EBITDA (17.6% margin)" -> ["Profitable", 0.8]
    Note: "'22: Burning $1m a month | Want to be breakeven '23" -> ["Not Profitable", 0.6]
    Note: "6m" -> ["Profitable", 0.8]
    Note: "12" -> ["Profitable", 0.9]
    """

    start_row, end_row = 4000, 4613 # Excel rows 1-2501 filled in Data_Extraction (1-2501 4o-mini, 2502-4615 4o)

    df_subset = df.iloc[start_row:end_row]
    # Apply the batch processing function
    revenue_values, revenue_confidences = process_notes(df_subset, 'Profitability Notes', task_description, operational_guidelines, examples)

    print('results: ', revenue_values, revenue_confidences)

    # Insert the values and confidences back into the DataFrame
    df.loc[start_row:end_row-1, 'Profitability Value'] = revenue_values
    df.loc[start_row:end_row-1, 'Profitability Confidence'] = revenue_confidences

    # Save the updated dataframe back to Excel
    output_path = './file_folder/Data_Extraction_Profitability_Edit.xlsx'
    df.to_excel(output_path, index=False)

    # Output path for user reference
    print(f"Updated Excel file saved to {output_path}")

main()

note:  ~20mm in EBITDA
API response:  [
    "Profitable",
    1.0
] to note:  ~20mm in EBITDA
clen [
    "Profitable",
    1.0
]
note:  $~35mm contribution margin
API response:  [
    "Profitable",
    1.0
] to note:  $~35mm contribution margin
clen [
    "Profitable",
    1.0
]
note:  25% Adj. EBITDA margin, but incremental standalone spend necessary at onset
API response:  ["Profitable", 1.0] to note:  25% Adj. EBITDA margin, but incremental standalone spend necessary at onset
clen ["Profitable", 1.0]
note:  burning 1 million
API response:  [
    "Not Profitable",
    0.9
] to note:  burning 1 million
clen [
    "Not Profitable",
    0.9
]
note:  Profitable now
API response:  ```json
["Profitable", 1.0]
``` to note:  Profitable now
clen ["Profitable", 1.0]
note:  Breakeven next year
API response:  [
  "Break Even",
  0.8
] to note:  Breakeven next year
clen [
  "Break Even",
  0.8
]
note:  nan
API response:  [
    "Blank",
    1.0
] to note:  nan
clen [
    "Blank",
    1.0
]
note:  